Install the dependencies:

In [1]:
!pip install transformers
!pip install diffusers
!pip install accelerate

!pip install flask-ngrok
!ngrok authtoken 'YOUR_NGROK_AUTH_TOKEN'

/bin/bash: line 1: ngrok: command not found


Initialize the SD model:

In [2]:
import torch
from diffusers import StableDiffusionPipeline

sd_pipe = StableDiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1",
    torch_dtype=torch.float16)

sd_pipe.to('cuda')

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.21.4",
  "_name_or_path": "stabilityai/stable-diffusion-2-1",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "requires_safety_checker": false,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "DDIMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

Create the Flask app:

In [3]:
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

In [4]:
import io
from flask import Flask, request, send_file, abort

@app.route('/text-to-image', methods=['POST', 'GET'])
def predict():
    if request.method in ('POST', 'GET'):
        prompt = request.get_json().get('prompt')

        if prompt and prompt.strip():
            image = sd_pipe(
                prompt,
                num_inference_steps=100).images[0]

            image_io = io.BytesIO()
            image.save(image_io, format='PNG')
            image_io.seek(0)
            return send_file(
                image_io,
                as_attachment=False,
                mimetype='image/png'
            )
        else:
            abort(500, description='Invalid prompt')

In [ ]:
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d2ae-35-230-66-89.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Oct/2023 04:16:43] "POST /text-to-image HTTP/1.1" 500 -


  0%|          | 0/100 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [01/Oct/2023 04:17:32] "POST /text-to-image HTTP/1.1" 200 -
